# Week 3: Probabilistic Reasoning

## Exercise 1: Balls and Urns

In this exercise, we will reproduce the computations of this week's first lecture, and extend them to $n$ urns.

First, we need to represent how many urns we have and how many balls of each type they contain.


In [10]:
import numpy as np
from scipy.stats import binom

urns = np.array([[5,3],[3,5]])
numUrns = urns.shape[0]
priors = np.ones(numUrns) / numUrns


As in the lecture, we will draw balls with replacement. The first thing to do is to be able to compute the probability of drawing $n$ white balls and $m$ blue balls, with replacement.

**Question 1** Implement a function called "likelihood" which takes as parameters
1. The index of the urn
1. The number of white balls drawn
1. The number of blue balls drawn
    
The distribution for this kind of observation is called the Binomial distribution. You can use the standard implementation of this distribution in the scipy.stats package for your implementation of this function.

In [11]:
# Your answer comes here
def likelihood(urnIndex,whiteDraws,blueDraws):
    whiteNum = urns[urnIndex,0]
    blueNum = urns[urnIndex,1]
    totalNum = whiteNum + blueNum
    
    p = whiteNum/totalNum
    k = whiteDraws
    n = whiteDraws+blueDraws    
    
    return binom.pmf(k,n,p)

print(likelihood(1,3,2))
print(likelihood(0,3,2))
print(likelihood(1,1,3))

0.20599365234374997
0.34332275390624994
0.3662109375000001


Now, let us compute the posterior probability of having used the first urn given the observed number of white and blue balls. 

**Question 2** Implement a function called "posterior" which takes the same parameters as the "likelihood" function. Try to make your implementation work for any number of urns.

What is the posterior probability of having picked urn 1 if we drew 3 white and 2 blue balls? What about 1 white and 3 blue balls?

In [12]:
# Your answer comes here
def posterior(urnIndex,whiteDraws,blueDraws):
    
    evidence = 0
    count = 0
    for p in priors:
        evidence += p*likelihood(count,whiteDraws,blueDraws) 
        count += 1          
    return priors[urnIndex]*likelihood(urnIndex,whiteDraws,blueDraws)/evidence   

print(posterior(0,3,2))  # 0.625
print(posterior(0,1,3))  
print(posterior(1,3,2))  # 0.375



0.625
0.2647058823529412
0.375


## Exercise 2: Entering college

Let's encode the probabilities of being smart, and an athlete and compute conditional probabilities. We're using the same probabilities as in the slides.


In [13]:
pS = 0.4
pA = 0.2
pC_SA = np.array([[ 0.91, 0.9],[0.9, 0.04]])

**Question 1** Implement a function that computes the joint probability of being smart (or not), entering college (or not) and being an athlete (or not). This functions takes three boolean parameters:
1. s, indicating whether the person is smart (True) or not Falss
1. c, indicating whether the person entered college
1. a, indicating whether the person is an athlete


In [14]:
# Your answer comes here
def jointProb(s,c,a):
    
    result = 0
    
    if c :
        if s  :
            if a  :
                result = pC_SA[0,0]*pS*pA # c,s,a
            else :
                result = pC_SA[0,1]*pS*(1-pA) # c,s, not a
        else:
            if a  :
                result = pC_SA[1,0]*(1-pS)*pA # c,not s, a
            else :
                result = pC_SA[1,1]*(1-pS)*(1-pA) # c,not s, not a               
         
    else:
        if s  :
            if a :
                result =(1 - pC_SA[0,0])*pS*pA # not c,s,a
            else :
                result =(1 - pC_SA[0,1])*pS*(1-pA) # not c, s, not a
        else:
            if a :
                result =(1 - pC_SA[1,0])*(1-pS)*pA # not c,not s, a
            else :
                result =(1 - pC_SA[1,1])*(1-pS)*(1-pA) # not c, not s, not a
        
    return result 

# lowercase: True, uppercase: False
psca = jointProb(True,True,True)
pscA = jointProb(True,True,False)
pSca = jointProb(False, True,True)
pScA = jointProb(False,True,False)
psCa = jointProb(True, False,True)
    

**Question 2** Let's compute the probability that somebody is smart given that that person is an athlete, using marginalisation:

$$ p(s|a) = \frac{p(s,a)}{p(a)} $$

where

$$p(s,a) = p(s,c,a) + p(s,\neg c, a)$$

In [15]:
# Your answer comes here

print((psca+psCa)/pA)

0.4000000000000001


**Question 3** Next, let's compute the probability that somebody is smart given that that person is in college

$$ P(s|c) = \frac{P(s,c)}{P(c)} = \frac{P(s,c,a) + P(s,c,\neg a)}{P(s,c,a)+P(s,c,\neg a)+P(\neg s,c,a) + P(\neg S,c,\neg a)}$$

In [16]:
# Your answer comes here

psc = psca + pscA
pc = psca+pscA+pSca+pScA

print(psca/pc)# correct reult: 0.7393442622950821

0.14918032786885246


**Question 4** Finally, compute the probability that a person is smart, given that this person is in college and an athlete.

In [17]:
# Your answer comes here
#p(s|c,a) = p(s,c,a)/p(c,a)
#         = p(s,c,a)/p(~s,c,a)+p(s,c,a)

print(psca/(pSca+psca)) #correct result: 0.40265486725663724

0.40265486725663724
